In [6]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_train.3.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_train.3.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_test.3.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_test.3.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.2.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.2.csv"]
txt_files =["/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/PAN2019_tweet_human1_chosen_set.2.csv"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))







txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/PAN2019_tweet_human1_chosen_set.2.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019textfiles_Type/PAN2019_tweet_human1_chosen_set.2.csv 5828
length of features used = 5828
len(all_training_text) 1440
len(X_train)= 1440 len(y_train)= 1440
len(all_test_text) 620
len(X_test= 620 len(y_test)= 620
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6887096774193548
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.682258064516129
****************************************************************
MLPClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8451612903225807
LogisticRegression****************************************************************
0.6951612903225807
SGDClassifier****************************************************************
0.6129032258064516
DecisionTreeClassifier****************************************************************
0.614516129032258
Naive****************************************************************
0.6661290322580645
SVM****************************************************************
0.6064516129032258
RandomForest_100****************************************************************
0.7903225806451613
RandomForest_200****************************************************************
0.785483870967742
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7612903225806451
BaggingClassifier****************************************************************
0.7161290322580646
AdaBoostClassifier****************************************************************
0.7241935483870968


In [16]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_train.3.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_train.3.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_test.3.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_test.3.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.2.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.2.csv"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivefs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200fs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifierfs=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.2.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.2.csv"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k5voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
manhattan_k13voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressionvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivevoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMvoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200voc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifiervoc=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.2.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.2.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.2.csv 6783
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.2.csv 7214
length of features used = 13997
len(all_training_text) 1440
len(X_train)= 1440 len(y_train)= 1440
len(all_test_text) 620
len(X_test= 620 len(y_test)= 620
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6483870967741936
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.6548387096774193
****************************************************

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.832258064516129
LogisticRegression****************************************************************
0.6951612903225807
SGDClassifier****************************************************************
0.5112903225806451
DecisionTreeClassifier****************************************************************
0.6564516129032258
Naive****************************************************************
0.7274193548387097
SVM****************************************************************
0.5758064516129032
RandomForest_100****************************************************************
0.7629032258064516
RandomForest_200****************************************************************
0.7838709677419354
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7806451612903226
BaggingClassifier****************************************************************
0.7612903225806451
AdaBoostClassifier****************************************************************
0.7370967741935484
txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv 71677
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv 67723
length of features used = 139400
len(all_training_text) 1440
len(X_train)= 1440 len(y_train)= 1440
len(all_test_text) 620
len(X_test= 620 len(y_test)= 620
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6645161290322581
****************

...

0.8290322580645161
LogisticRegression****************************************************************
0.7112903225806452
SGDClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.7048387096774194
DecisionTreeClassifier****************************************************************
0.6548387096774193
Naive****************************************************************
0.7274193548387097
SVM****************************************************************
0.635483870967742
RandomForest_100****************************************************************
0.7693548387096775
RandomForest_200****************************************************************
0.785483870967742
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7806451612903226
BaggingClassifier****************************************************************
0.7048387096774194
AdaBoostClassifier****************************************************************
0.7370967741935484


In [21]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= MLPClassifierfs 
y= manhattan_k5fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= MLPClassifierfs 
y= manhattan_k13fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= MLPClassifierfs 
y= MLPClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= MLPClassifierfs 
y= LogisticRegressionfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= MLPClassifierfs 
y= SGDClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= MLPClassifierfs 
y= DecisionTreeClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= MLPClassifierfs 
y= Naivefs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= MLPClassifierfs 
y= SVMfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= MLPClassifierfs 
y= RandomForest_100fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= MLPClassifierfs 
y= RandomForest_200fs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= MLPClassifierfs 
y= XGBClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= MLPClassifierfs 
y= BaggingClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= MLPClassifierfs 
y= AdaBoostClassifierfs

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)








The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 142 negative = 28
manhattan_k=13************************************ ****************************
positive= 139 negative = 29
MLPClassifier****************************************************************
positive= 0 negative = 0
LogisticRegression****************************************************************
positive= 118 negative = 33
SGDClassifier****************************************************************
positive= 245 negative = 46
DecisionTreeClassifier****************************************************************
positive= 152 negative = 43
Naive****************************************************************
positive= 98 negative = 33
SVM****************************************************************
positive= 203 negative = 44
RandomForest_100************************************************************

In [22]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= MLPClassifiervoc 
y= manhattan_k5voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= MLPClassifiervoc 
y= manhattan_k13voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= MLPClassifiervoc 
y= MLPClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= MLPClassifiervoc 
y= LogisticRegressionvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= MLPClassifiervoc 
y= SGDClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= MLPClassifiervoc 
y= DecisionTreeClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= MLPClassifiervoc 
y= Naivevoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= MLPClassifiervoc 
y= SVMvoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= MLPClassifiervoc 
y= RandomForest_100voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= MLPClassifiervoc 
y= RandomForest_200voc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= MLPClassifiervoc 
y= XGBClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= MLPClassifiervoc 
y= BaggingClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= MLPClassifiervoc 
y= AdaBoostClassifiervoc

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)









The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 140 negative = 38
manhattan_k=13************************************ ****************************
positive= 129 negative = 32
MLPClassifier****************************************************************
positive= 0 negative = 0
LogisticRegression****************************************************************
positive= 107 negative = 34
SGDClassifier****************************************************************
positive= 106 negative = 29
DecisionTreeClassifier****************************************************************
positive= 148 negative = 40
Naive****************************************************************
positive= 98 negative = 35
SVM****************************************************************
positive= 153 negative = 33
RandomForest_100************************************************************

In [18]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_train.3.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_train.3.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_test.3.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_test.3.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.1.csv"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
manhattan_k5tfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
manhattan_k13tfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
MLPClassifiertfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
LogisticRegressiontfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
SGDClassifiertfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
DecisionTreeClassifiertfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
Naivetfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVMtfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_100tfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
RandomForest_200tfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
XGBClassifiertfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
BaggingClassifiertfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
AdaBoostClassifiertfidf=([(i,j) for i, j in zip(y_pred, y_test)])
print(metrics.accuracy_score(y_test, y_pred))





txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.1.csv 3584
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.1.csv 3710
length of features used = 7294
len(all_training_text) 1440
len(X_train)= 1440 len(y_train)= 1440
len(all_test_text) 620
len(X_test= 620 len(y_test)= 620
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6451612903225806
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.6629032258064517
*****************************************************

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8338709677419355
LogisticRegression****************************************************************
0.6951612903225807
SGDClassifier****************************************************************
0.6967741935483871
DecisionTreeClassifier****************************************************************
0.6306451612903226
Naive****************************************************************
0.7225806451612903
SVM****************************************************************
0.5758064516129032
RandomForest_100****************************************************************
0.7693548387096775
RandomForest_200****************************************************************
0.7919354838709678
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7806451612903226
BaggingClassifier****************************************************************
0.7306451612903225
AdaBoostClassifier****************************************************************
0.7370967741935484


In [3]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_train.3.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_train.3.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_test.3.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_test.3.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.1.csv"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("mean length = ",(len([x for item in all_training_text for x in item]))/len(all_training_text))
print("len(all_training_text)",len(all_training_text))
X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(y_test, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
y_pred = Naive.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))






txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv 71677
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv 67723
length of features used = 139400
mean length =  2066.742361111111
len(all_training_text) 1440
len(X_train)= 1440 len(y_train)= 1440
len(all_test_text) 620
len(X_test= 620 len(y_test)= 620
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6645161290322581
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.6725806451612903
******

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.8225806451612904
LogisticRegression****************************************************************
0.7112903225806452
SGDClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5209677419354839
DecisionTreeClassifier****************************************************************
0.6661290322580645
Naive****************************************************************
0.7274193548387097
SVM****************************************************************
0.635483870967742
RandomForest_100****************************************************************
0.7790322580645161
RandomForest_200****************************************************************
0.7887096774193548
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7806451612903226
BaggingClassifier****************************************************************
0.7016129032258065
AdaBoostClassifier****************************************************************
0.7370967741935484


In [26]:
import sys
sys.path.append("/anaconda3/lib/python3.7/site-packages")
import numpy as np
import numpy
import nltk
import pickle
import pandas as pd
import glob
import os
import re
import operator
from collections import Counter
#from read_xml_files import *
##from s_stemmer import *
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from itertools import groupby
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import xml.etree.ElementTree as ET
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
##from pattern.text.en import singularize
##nltk.download('vader_lexicon')
##nltk.download('punkt')
#import the relevant modules from the NLTK library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
##tt = ToktokTokenizer()#keeps the url as is...................................................
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
Porter_stemmer = PorterStemmer()
Lancaster_stemmer = LancasterStemmer()
WordNet_lemmatizer = WordNetLemmatizer()
#nltk.download('averaged_perceptron_tagger')
##Train_path="D:/Data/PAN2015-Profiling/pan15-author-profiling-training-dataset-english-2015-04-23"#path for train data
##Test_path="D:/Data/PAN2015-Profiling/pan-ap2015-test/en" #path for test data

##Train_path="D:/NLP/PAN2014/pan14-author-profiling-training-corpus-english-twitter-2014-04-16"#path for train data D:\Data\PAN2013-Profiling\Training\en  D:\NLP\PAN2014
##train_truth_path="D:/NLP/PAN2014/truth_train"
##
##test_truth_path="D:/NLP/PAN2014/pan14_test"
##Test_path="D:/NLP/PAN2014/pan14_test/en"
##Content = open('results_of_clssification_all.txt', 'w')

M_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_train.3.txt"
F_train_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_train.3.txt"

M_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_test.3.txt"
F_test_file= "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_test.3.txt"

def all_txt(M_txt_file,  F_txt_file):
    with open(M_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            male_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in male_dict:
                # append the new number to the existing array at this slot
                    male_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    male_dict[line[1]] = [line[3]]
    male_content= [("M", (' '.join(a)).split()) for a in list(male_dict.values())]
  
    with open(F_txt_file, mode="r", encoding="utf-8") as f:
            txt_list = [(line[:-1]).split("\t") for line in f]
            female_dict = dict()
            content=[]
            for line in txt_list:
                if line[1] in female_dict:
                # append the new number to the existing array at this slot
                    female_dict[line[1]].append(line[3])
                else:
                # create a new array in this slot
                    female_dict[line[1]] = [line[3]]
    female_content= [("F", (' '.join(a)).split()) for a in list(female_dict.values())]
    all_txt_per_person = male_content + female_content
    return all_txt_per_person

##print( [((all_txt(M_txt_file,  F_txt_file))[i][0])  for i in range(len((all_txt(M_txt_file,  F_txt_file))))]) helps to determine the gender....

      
"""
create feature vector from the training set.....................................................................................
"""

def WordFeatures(word_list, all_training_text):

    fvs_words = np.array([[author.count(word) for word in word_list] for author in all_training_text]).astype(np.float64)

    # normalise by dividing each row by number of tokens for each author........
    fvs_words /= np.c_[np.array([len(author) for author in all_training_text])]

    return fvs_words



"""
From the saved csv file, recover the saved features to be used...............................................................
"""
import csv
word_list=[]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.1M.txt", "/Users/catherine/Desktop/NLP/PAN Datasets/PAN_chosen_tokens/2014/output2014OR.2F.txt"]
txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv"]
##txt_files =[ "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_chosen_set.1.csv", "/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_chosen_set.1.csv"]

print("txt_files of features used",txt_files)
##for txt_file in txt_files:
##    with open(txt_file, mode="r", encoding="utf-8") as f:
##        reader = csv.reader(f, delimiter=",")
##        next(reader) # skip header
##        word_list1 =  ([(r.split(":"))[1] for r in f])
##        print(txt_file, len(word_list1))
##        word_list = word_list + word_list1
##print("length of features used =", len(word_list))

for txt_file in txt_files:
    with open(txt_file, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",")
        next(reader) # skip header
        word_list1 =  [r[0] for r in reader]
        print(txt_file, len(word_list1))
        word_list = word_list + word_list1
print("length of features used =", len(word_list))


"""
Prepare the training and test sets to be parsed to the classifies............................................................
"""
A=(all_txt(M_train_file,  F_train_file))
all_training_text = ( [(A[i][1])  for i in range(len(A))])
print("mean length = ",(len([x for item in all_training_text for x in item]))/len(all_training_text))
print("len(all_training_text)",len(all_training_text))
#X_train=WordFeatures(word_list, all_training_text)
y_train=np.array( [(A[i][0])  for i in range(len(A))])
#print("len(X_train)=",len(X_train), "len(y_train)=", len(y_train))

B=(all_txt(M_test_file,  F_test_file))
all_test_text = ( [(B[i][1])  for i in range(len(B))])
print("len(all_test_text)",len(all_test_text))
#X_test = WordFeatures(word_list, all_test_text)
y_test = np.array( [(B[i][0])  for i in range(len(B))])
#print("len(X_test=",len(X_test), "len(y_test)=",len(y_test))


X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)




#..................................................................................

data_tags = ["type","gender","tweet"]
train_data_list = []
i=0
for f in y_train:
   
    train_data_list.append((f, y_train[i],all_training_text[i]))
    i += 1
# We have training data available as dictionary filename, category, data
train_data = pd.DataFrame.from_records(train_data_list, columns=data_tags)




#..................................................................................

test_data_list = []
ii=0
for f in y_test:
   
    test_data_list.append((f, y_test[ii], all_test_text[ii]))
    ii += 1
# We have training data available as dictionary filename, category, data
test_data = pd.DataFrame.from_records(test_data_list, columns=data_tags)

#..................................................................................


import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

# :Tokenize and Prepare Vocabulary

train_posts = train_data['tweet']
train_tags = train_data['type']

test_posts = test_data['tweet']
test_tags = test_data['type']


num_labels = 2
vocab_size = 1000#len(word_list)#most common number of words will be then kept for use in the vector
#vocab_size = 1000#by changing the vocubulary size... acc count_type =  0.932258064516129#acc count_gender = 0.8298387096774194
#vocab_size = 100#acc count_type =0.9274193548387096 acc count_gender = 0.792741935483871
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)#IF I USE OTHER TOKENIZER...
tokenizer.fit_on_texts(train_posts)
#print(tokenizer.word_index)
#print((tokenizer.word_counts))#provides a dictionary of the words and the count......................
#sorted_x = sorted((tokenizer.word_counts).items(), key=operator.itemgetter(1),reverse=True)
#print(sorted_x)
#print((tokenizer.document_count))#number of documents..............
#print((tokenizer.word_docs))#provides a dictionary of the words and the number of documents they appear in......................
print('Found %d unique words.' % len(tokenizer.word_index))#shows total vocubulary of the text dataset
x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')#WHAT HAPPENS WHEN I GIVE IT THE HAND CRAFTED TOKENS??
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')

##--------what about applying knn at this point-------------------------------
##--------what about changing the vocubulary size-------------------------------works well with reduced sized
##----------One popular method for hyperparameter optimization is grid search.-----------
##-----determine the best set of parameters with the highest accuracy..........this is for the keras model------


      
encoder = LabelBinarizer()


encoder.fit(train_tags)

y_train = encoder.transform(train_tags)#same as the y train generated with my model
y_train = np.hstack((y_train, 1 - y_train))#used for two label cases........

y_test = encoder.transform(test_tags)
y_test = np.hstack((y_test, 1 - y_test))


"""
fit_on_texts Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t. word_index["the"] = 0; word_index["cat"] = 1 it is word -> index dictionary so every word gets a unique integer value. So lower integer means more frequent word (often the first few are punctuation because they appear a lot).
texts_to_sequences Transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.
CLEAN TEXT TO THE DESIRED LEVEL AND USE KERAS INBUILF TFIDF TO GENERATE A MATRIX....
"""
from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
#8. manhattan K-Nearest Neighbours..............................................................................

k=5

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=5************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k5tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("************************************ ****************************")

k=13

print(k)
"""
8. manhattan K-Nearest Neighbours..............................................................................
"""
print("manhattan_k=13************************************ ****************************")
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = k, metric= "manhattan")
knn.fit(x_train, train_tags)
y_pred = knn.predict(x_test)
manhattan_k13tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print("manhattan K-Nearest Neighbours")
print(metrics.accuracy_score(test_tags, y_pred))
print("****************************************************************")


from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier


print("MLPClassifier****************************************************************")
clf = MLPClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
MLPClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("LogisticRegression****************************************************************")
clf = LogisticRegression()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
LogisticRegressiontfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SGDClassifier****************************************************************")
clf = SGDClassifier()
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
SGDClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("DecisionTreeClassifier****************************************************************")
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()
# Train Decision Tree Classifer
clf = clf.fit(x_train, train_tags)
#Predict the response for test dataset
y_pred = clf.predict(x_test)
DecisionTreeClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("Naive****************************************************************")
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train, train_tags)
y_pred = Naive.predict(x_test)
Naivetfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("SVM****************************************************************")
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train, train_tags)
y_pred = SVM.predict(x_test)
SVMtfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("RandomForest_100****************************************************************")
clf=RandomForestClassifier(n_estimators=100)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_100tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

print("RandomForest_200****************************************************************")
clf=RandomForestClassifier(n_estimators=200)
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,train_tags)
y_pred=clf.predict(x_test)
RandomForest_200tfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))

from xgboost.sklearn import XGBClassifier 
print("XGBClassifier****************************************************************")
clf=XGBClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
XGBClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))



print("BaggingClassifier****************************************************************")
from sklearn.ensemble.bagging import BaggingClassifier
clf=BaggingClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
BaggingClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))


print("AdaBoostClassifier****************************************************************")
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
clf=AdaBoostClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train, train_tags)
y_pred=clf.predict(x_test)
AdaBoostClassifiertfidf=([(i,j) for i, j in zip(y_pred, test_tags)])
print(metrics.accuracy_score(test_tags, y_pred))




from keras.models import *
from keras.layers import *

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))#512 neurons in the first hidden layer
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
#model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)



#Evaluate model.............................
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])









txt_files of features used ['/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv', '/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv']
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_female_vocubulary.1.csv 71677
/Users/catherine/Desktop/NLP/PAN_Datasets/PAN2019/PAN_2019_txtfiles/PAN2019_tweet_male_ vocubulary_set.1.csv 67723
length of features used = 139400
mean length =  2066.742361111111
len(all_training_text) 1440
len(all_test_text) 620
Found 115144 unique words.
5
manhattan_k=5************************************ ****************************
manhattan K-Nearest Neighbours
0.6645161290322581
************************************ ****************************
13
manhattan_k=13************************************ ****************************
manhattan K-Nearest Neighbours
0.6451612903225806
**************************************************

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6483870967741936
Naive****************************************************************
0.8048387096774193
SVM****************************************************************
0.7693548387096775
RandomForest_100****************************************************************
0.782258064516129
RandomForest_200****************************************************************
0.8016129032258065
XGBClassifier****************************************************************


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7919354838709678
BaggingClassifier****************************************************************
0.7483870967741936
AdaBoostClassifier****************************************************************
0.7467741935483871
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_8 (Activation)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 1026      
____________________________________________________

In [27]:
print("The sign test...****************************************************************")
print("manhattan_k=5************************************ ****************************")
x= Naivetfidf 
y= manhattan_k5tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)
        
print("manhattan_k=13************************************ ****************************")
x= Naivetfidf 
y= manhattan_k13tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)

print("MLPClassifier****************************************************************")
x= Naivetfidf 
y= MLPClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("LogisticRegression****************************************************************")
x= Naivetfidf 
y= LogisticRegressiontfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("SGDClassifier****************************************************************")
x= Naivetfidf 
y= SGDClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("DecisionTreeClassifier****************************************************************")
x= Naivetfidf 
y= DecisionTreeClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("Naive****************************************************************")
x= Naivetfidf 
y= Naivetfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("SVM****************************************************************")
x= Naivetfidf 
y= SVMtfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_100****************************************************************")
x= Naivetfidf 
y= RandomForest_100tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("RandomForest_200****************************************************************")
x= Naivetfidf 
y= RandomForest_200tfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("XGBClassifier****************************************************************")
x= Naivetfidf 
y= XGBClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




print("BaggingClassifier****************************************************************")
x= Naivetfidf 
y= BaggingClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)



print("AdaBoostClassifier****************************************************************")
x= Naivetfidf 
y= AdaBoostClassifiertfidf

positive=0
negative=0
for i in range(len(x)):
    if (x[i]!=y[i]):
        if (x[i][0]==x[i][1] and y[i][0]!=y[i][1]):
            positive=positive+1
        else: 
            negative=negative+1
print("positive=",positive, "negative =" , negative)




The sign test...****************************************************************
manhattan_k=5************************************ ****************************
positive= 154 negative = 67
manhattan_k=13************************************ ****************************
positive= 172 negative = 73
MLPClassifier****************************************************************
positive= 57 negative = 45
LogisticRegression****************************************************************
positive= 73 negative = 56
SGDClassifier****************************************************************
positive= 67 negative = 33
DecisionTreeClassifier****************************************************************
positive= 146 negative = 49
Naive****************************************************************
positive= 0 negative = 0
SVM****************************************************************
positive= 79 negative = 57
RandomForest_100***************************************************************